In [20]:
!pip install --upgrade pip
!pip install opencv-python

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages (19.0.1)
    100% |████████████████████████████████| 25.4MB 2.1MB/s eta 0:00:01


In [5]:
import argparse
import copy
import numpy as np
import os

import chainer
from chainer.datasets import ConcatenatedDataset
from chainer.datasets import TransformDataset
from chainer.optimizer import WeightDecay
from chainer import serializers
from chainer import training
from chainer.training import extensions
from chainer.training import triggers

from chainercv.datasets import voc_bbox_label_names
from chainercv.links.model.ssd import GradientScaling
from chainercv.links.model.ssd import multibox_loss
from chainercv.links import SSD300
from chainercv.links import SSD512
from chainercv import transforms

from chainercv.links.model.ssd import random_crop_with_bbox_constraints
from chainercv.links.model.ssd import random_distort
from chainercv.links.model.ssd import resize_with_random_interpolation

from datasetfromdat import DatasetFromDat
from via_utils import via_bbox_label_names
from via_detection_evaluator import DetectionEvaluator

class MultiboxTrainChain(chainer.Chain):

    def __init__(self, model, alpha=1, k=3):
        super(MultiboxTrainChain, self).__init__()
        with self.init_scope():
            self.model = model
        self.alpha = alpha
        self.k = k

    def __call__(self, imgs, gt_mb_locs, gt_mb_labels):
        mb_locs, mb_confs = self.model(imgs)
        loc_loss, conf_loss = multibox_loss(
            mb_locs, mb_confs, gt_mb_locs, gt_mb_labels, self.k)
        loss = loc_loss * self.alpha + conf_loss

        chainer.reporter.report(
            {'loss': loss, 'loss/loc': loc_loss, 'loss/conf': conf_loss},
            self)

        return loss
    

class Transform(object):

    def __init__(self, coder, size, mean):
        # to send cpu, make a copy
        self.coder = copy.copy(coder)
        self.coder.to_cpu()

        self.size = size
        self.mean = mean

    def __call__(self, in_data):
        # There are five data augmentation steps
        # 1. Color augmentation
        # 2. Random expansion
        # 3. Random cropping
        # 4. Resizing with random interpolation
        # 5. Random horizontal flipping

        img, bbox, label = in_data

        # 1. Color augmentation
        img = random_distort(img)

        # 2. Random expansion
        if np.random.randint(2):
            img, param = transforms.random_expand(
                img, fill=self.mean, return_param=True)
            bbox = transforms.translate_bbox(
                bbox, y_offset=param['y_offset'], x_offset=param['x_offset'])

        # 3. Random cropping
        img, param = random_crop_with_bbox_constraints(
            img, bbox, return_param=True)
        bbox, param = transforms.crop_bbox(
            bbox, y_slice=param['y_slice'], x_slice=param['x_slice'],
            allow_outside_center=False, return_param=True)
        label = label[param['index']]

        # 4. Resizing with random interpolatation
        _, H, W = img.shape
        img = resize_with_random_interpolation(img, (self.size, self.size))
        bbox = transforms.resize_bbox(bbox, (H, W), (self.size, self.size))

        # 5. Random horizontal flipping
        img, params = transforms.random_flip(
            img, x_random=True, return_param=True)
        bbox = transforms.flip_bbox(
            bbox, (self.size, self.size), x_flip=params['x_flip'])

        # Preparation for SSD network
        img -= self.mean
        mb_loc, mb_label = self.coder.encode(bbox, label)

        return img, mb_loc, mb_label



In [22]:
model = SSD300(
    n_fg_class=len(voc_bbox_label_names),
    pretrained_model='voc0712')

model.use_preset('evaluate')
train_chain = MultiboxTrainChain(model)

#chainer.cuda.get_device_from_id(0).use()
#model.to_gpu()

train = TransformDataset(
    DatasetFromDat(file_path = 'Rack.dat'),
    Transform(model.coder, model.insize, model.mean))
train_iter = chainer.iterators.MultiprocessIterator(train, 6)

test = DatasetFromDat('Rack_val.dat')
test_iter = chainer.iterators.SerialIterator(
    test, 6, repeat=False, shuffle=False)

# initial lr is set to 1e-3 by ExponentialShift
optimizer = chainer.optimizers.MomentumSGD()
optimizer.setup(train_chain)
for param in train_chain.params():
    if param.name == 'b':
        param.update_rule.add_hook(GradientScaling(2))
    else:
        param.update_rule.add_hook(WeightDecay(0.0005))
updater = training.StandardUpdater(train_iter, optimizer, device=-1)
# 120000->8000
trainer = training.Trainer(updater, (500, 'iteration'), 'result')
# 80000->5000,100000->7000
trainer.extend(
    extensions.ExponentialShift('lr', 0.1, init=1e-3),
    trigger=triggers.ManualScheduleTrigger([300, 400], 'iteration'))
# 10000->700
trainer.extend(
    DetectionEvaluator(
        test_iter, model, use_07_metric=True,
        label_names=via_bbox_label_names),
    trigger=(7, 'iteration'))
log_interval = 10, 'iteration'
trainer.extend(extensions.LogReport(trigger=log_interval))
trainer.extend(extensions.observe_lr(), trigger=log_interval)
trainer.extend(extensions.PrintReport(
    ['epoch', 'iteration', 'lr',
     'main/loss', 'main/loss/loc', 'main/loss/conf',
     'validation/main/map']),
    trigger=log_interval)
trainer.extend(extensions.ProgressBar(update_interval=10))
# 10000->700
trainer.extend(extensions.snapshot(), trigger=(50, 'iteration'))
# 120000->8000
trainer.extend(
    extensions.snapshot_object(model, 'model_iter_{.updater.iteration}'),
    trigger=(500, 'iteration'))

In [ ]:
trainer.run()
serializers.save_npz('via_model.npz', model)
serializers.save_npz('via_state.npz', optimizer)


/home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/chainercv/transforms/image/resize.py:33: RuntimeWarning: cv2 is not installed on your environment. ChainerCV will fall back on Pillow. Installation of cv2 is recommended for faster computation. 
  RuntimeWarning)


epoch       iteration   lr          main/loss   main/loss/loc  main/loss/conf  validation/main/map
0           10          0.001       7.47845     2.28332        5.19514         0.0132312            
     total [#.................................................]  2.00%
this epoch [##########........................................] 20.00%
        10 iter, 0 epoch / 500 iterations
       inf iters/sec. Estimated time to finish: 0:00:00.
0           20          0.001       7.2191      2.4708         4.7483          0.0999519            
     total [##................................................]  4.00%
this epoch [####################..............................] 40.00%
        20 iter, 0 epoch / 500 iterations
   0.12944 iters/sec. Estimated time to finish: 1:01:48.273239.
0           30          0.001       7.20564     2.64248        4.56316         0.0321403            
     total [###...............................................]  6.00%
this epoch [##########################

In [ ]:
# 14:33 1:01:48.273239.
# 14:37 1:21:57.626593.
# 14:41 1:16:58.949802.
# 14:46 1:50:06.859916.
# 14:51 2:01:28.154509.
# 14:57 2:09:45.174329.
# 15:04 2:26:51.703056.